In [1]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dropout, Dense, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.backend import clear_session

import numpy as np
import matplotlib.pyplot as plt
from transformer import TransformerBlock, TokenAndPositionEmbedding
from utils import load_dataset, train

In [3]:
X_train, y_train, X_test, y_test = load_dataset('decoded')

In [11]:
sample_size = 4 # A C G T
embed_dim = 8 # Embedding size for each token
num_heads = 4 # Number of attention heads
ff_dim = 32 # Hidden layer size of feed forward inside transformer
max_len = 1000

clear_session()

inputs = Input(shape=(max_len,))
x = TokenAndPositionEmbedding(max_len, sample_size, embed_dim)(inputs)
x = TransformerBlock(embed_dim, num_heads, ff_dim)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.2)(x)
outputs = Dense(31, activation='relu')(x)

model = Model(inputs, outputs)

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1000)]            0         
_________________________________________________________________
token_and_position_embedding (None, 1000, 8)           8032      
_________________________________________________________________
transformer_block (Transform (None, 1000, 8)           872       
_________________________________________________________________
dense_6 (Dense)              (None, 1000, 64)          576       
_________________________________________________________________
dropout_2 (Dropout)          (None, 1000, 64)          0         
_________________________________________________________________
dense_7 (Dense)              (None, 1000, 31)          2015      
Total params: 11,495
Trainable params: 11,495
Non-trainable params: 0
_________________________________________________________

In [4]:
es = EarlyStopping(monitor='loss', patience=15, verbose=1)
optimizer = Adam(lr=1e-3)

epochs = 1500
validation_freq = 5
boundaries = [100, 300, 500, 1000, 2500, 5000]

In [12]:
%%time
model = train(dataset=(X_train, y_train, X_test, y_test),
              model=model,
              epochs=epochs,
              verbose=1,
              validation_freq=validation_freq,
              optimizer=optimizer,
              callbacks=[es])


Epoch 1/1500


ValueError: in user code:

    /home/victor/miniconda3/envs/genetics/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:805 train_function  *
        return step_function(self, iterator)
    /home/victor/miniconda3/envs/genetics/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:795 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/victor/miniconda3/envs/genetics/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/victor/miniconda3/envs/genetics/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/victor/miniconda3/envs/genetics/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3417 _call_for_each_replica
        return fn(*args, **k